In [1]:
#homework 3

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

1. Modeling Volatility and VaR

In [23]:
df = pd.read_excel('../data/spy_data.xlsx', sheet_name='total returns')
df['Excess Returns'] = df['SPY'] - df['^IRX']
df

,date,SPY,^IRX,Excess Returns
0,1994-01-04,0.004035,0.000122,0.003913
1,1994-01-05,0.002009,0.000122,0.001888
2,1994-01-06,0.000000,0.000121,-0.000121
3,1994-01-07,0.006016,0.000119,0.005897
4,1994-01-10,0.011960,0.000119,0.011841
...,...,...,...,...
7675,2024-07-29,0.000588,0.000204,0.000384
7676,2024-07-30,-0.005066,0.000204,-0.005270
7677,2024-07-31,0.016255,0.000204,0.016051
7678,2024-08-01,-0.014161,0.000203,-0.014364


1.1 Historic VaR

In [27]:
df = df[df['date'] >= '2000-12-29']
confidence_level = .95

def calc_var(series, confidence_level):
    return np.percentile(series,(1-confidence_level)*100)

df['Expanding VaR'] = df['Excess Returns'].expanding().apply(lambda x: calc_var(x,confidence_level),raw=True)

df

,date,SPY,^IRX,Excess Returns,Expanding VaR
1752,2000-12-29,-0.018929,0.000227,-0.019157,-0.019157
1753,2001-01-02,-0.018104,0.000225,-0.018329,-0.019115
1754,2001-01-03,0.048035,0.000219,0.047815,-0.019074
1755,2001-01-04,-0.010764,0.000208,-0.010972,-0.019033
1756,2001-01-05,-0.032643,0.000197,-0.032840,-0.030103
...,...,...,...,...,...
7675,2024-07-29,0.000588,0.000204,0.000384,-0.018775
7676,2024-07-30,-0.005066,0.000204,-0.005270,-0.018775
7677,2024-07-31,0.016255,0.000204,0.016051,-0.018774
7678,2024-08-01,-0.014161,0.000203,-0.014364,-0.018774
